In [2]:
import os
import pandas as pd
import numpy as np

In [53]:
os.chdir("../../tests/geneticdata")
files = os.listdir()
for i, f in enumerate(files):
    print(i, f)

0 GeneticData_-_3.txt
1 GeneticData_-_2.txt
2 GeneticData_-_1.txt
3 GeneticData_-_5.txt
4 GeneticData_-_4.txt
5 GeneticData_-_6.txt
6 GeneticData_-_7.txt
7 rename_files.py
8 GeneticData_-_9.txt
9 GeneticData_-_8.txt


In [145]:
input_file = files[4]
chromo_name = "mtDNA"
output_file = "output_file.tsv"

with open(input_file, "rb") as f:
    i = 0
    chromo_line = None
    data_list = []
    while True:
        line_b = f.readline()
        line = str(line_b)
        if chromo_name in line:
            chromo_line = i + 1
        if i == chromo_line:
            data_list.append(line[2:line.find("\\")])
        i += 1
        if not line_b:
            break

data = np.array(data_list)

seq_lens = []
for seq in data:
    seq_lens.append(len(seq))
    
if len(np.unique(seq_lens)) > 1:
    # TODO: throw error
    pass
else:
    positions = seq_lens[0]

with open(output_file, "w") as f:
    f.write("\t".join(output_header) + "\n")
    
for p in range(positions):
    nucleotides = []
    for seq in data:
        if seq[p].upper() in "ACGT":
            nucleotides.append(seq[p].upper())
    if nucleotides.count(nucleotides[0]) < len(nucleotides):
        alleles = set(nucleotides)
        freqs = {}
        for a in alleles:
            freqs[a] = nucleotides.count(a)
        ma = max(freqs, key=freqs.get)
        mi = min(freqs, key=freqs.get)
        maf = freqs[mi] / sum(freqs[a] for a in alleles)
        row = "\t".join([chromo_name, str(p), "/".join(alleles), str(ma), str(mi), str(round(maf, 2))]) + "\n"
        with open(output_file, "a") as f:
            f.write(row)

In [15]:
x = [1, 1, 1, 2]
x = set(x)
print(len(x))

2


In [36]:
x = "A/AaaaaaA/Aa"
i = len(x) - x[::-1].find("/")
print(x[i:])
print(os.getcwd())

Aa
/Users/otodreas/Lund/Courses/MiscExercisesBINP16/RunningExercise3_BINP16/src/partiiiprograms


In [15]:
x = {'A': 10, 'B': 3, 'C': 3}
ma = max(x, key=x.get)
mi = min(x, key=x.get)
print(ma, mi)
print(list(x.values()).count(min(x.values())))
#for val in [ma, mi]:
    #key = x.get(val)
 #   print(key)

A B
2


In [17]:
x = {'A': 10, 'B': 3, 'C': 3}
ma = max(x, key=x.get)
mi = min(x, key=x.get)

import random

ma = []
mi = []
for item in x.items():
    if item[1] == min(list(x.values())):
        mi.append(item[0])
    elif item[1] == max(list(x.values())):
        ma.append(item[0])
ma = random.choice(ma)
mi = random.choice(mi)

('A', 10)
('B', 3)
('C', 3)


In [26]:
random.seed(0)
print(type(random.choice(['B'])))

<class 'str'>


In [36]:
x = ['a', 'b', 'c']
popper = 'd'
x.pop(x.index(popper)) if popper in x else x
print(x)

['a', 'b', 'c']
